In [16]:
from __future__ import division
from spellingCorrector import correctSpelling
import numpy as np
import pandas as pd
import utils
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
ls ../data

attributes.csv                relevance_instructions.docx  train.csv
attributes.csv.zip            sample_submission.csv.zip    train.csv.zip
product_descriptions.csv      test.csv
product_descriptions.csv.zip  test.csv.zip


In [18]:
train_data_path ="../data/train.csv"
attributes_path = "../data/attributes.csv"
product_description_path = "../data/product_descriptions.csv"
test_data_path = "../data/test.csv"

In [19]:
train_data = pd.read_csv(train_data_path)
train_data.head()
train_data = train_data[(train_data.relevance != 2.50) & (train_data.relevance != 2.25) & (train_data.relevance != 2.75) & (train_data.relevance != 1.75) & (train_data.relevance != 1.50) & (train_data.relevance != 1.25)]

In [20]:
pd.value_counts(train_data.relevance)

3.00    19125
2.33    16060
2.67    15202
2.00    11730
1.67     6780
1.33     3006
1.00     2105
dtype: int64

In [21]:
test_data = pd.read_csv(test_data_path)
test_data.head()

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668


In [22]:
attributes = pd.read_csv(attributes_path, nrows=10000)
attributes.head()

,product_uid,name,value
0,100001,Bullet01,Versatile connector for various 90° connection...
1,100001,Bullet02,Stronger than angled nailing or screw fastenin...
2,100001,Bullet03,Help ensure joints are consistently straight a...
3,100001,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,100001,Bullet05,Made from 12-Gauge steel


In [23]:
productDescription = pd.read_csv(product_description_path)
productDescription.head()

,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they ..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...
2,100003,Classic architecture meets contemporary design...
3,100004,The Grape Solar 265-Watt Polycrystalline PV So...
4,100005,Update your bathroom with the Delta Vero Singl...


In [24]:
productDescription[1:100].product_description.values

array([ 'BEHR Premium Textured DECKOVER is an innovative solid color coating. It will bring your old, weathered wood or concrete back to life. The advanced 100% acrylic resin formula creates a durable coating for your tired and worn out deck, rejuvenating to a whole new look.  For the best results, be sure to properly prepare the surface using other applicable BEHR products displayed above.California residents: see&nbsp;Proposition 65 informationRevives wood and composite decks, railings, porches and boat docks, also great for concrete pool decks, patios and sidewalks100% acrylic solid color coatingResists cracking and peeling and conceals splinters and cracks up to 1/4 in.Provides a durable, mildew resistant finishCovers up to 75 sq. ft. in 2 coats per gallonCreates a textured, slip-resistant finishFor best results, prepare with the appropriate BEHR product for your wood or concrete surfaceActual paint colors may vary from on-screen and printer representationsColors available to be ti

In [13]:
productDescription[1:2].product_description.values

array([ 'BEHR Premium Textured DECKOVER is an innovative solid color coating. It will bring your old, weathered wood or concrete back to life. The advanced 100% acrylic resin formula creates a durable coating for your tired and worn out deck, rejuvenating to a whole new look.  For the best results, be sure to properly prepare the surface using other applicable BEHR products displayed above.California residents: see&nbsp;Proposition 65 informationRevives wood and composite decks, railings, porches and boat docks, also great for concrete pool decks, patios and sidewalks100% acrylic solid color coatingResists cracking and peeling and conceals splinters and cracks up to 1/4 in.Provides a durable, mildew resistant finishCovers up to 75 sq. ft. in 2 coats per gallonCreates a textured, slip-resistant finishFor best results, prepare with the appropriate BEHR product for your wood or concrete surfaceActual paint colors may vary from on-screen and printer representationsColors available to be ti

In [14]:
pd.value_counts(train_data.relevance)

3.00    19125
2.33    16060
2.67    15202
2.00    11730
1.67     6780
1.33     3006
1.00     2105
2.50       19
2.25       11
2.75       11
1.75        9
1.50        5
1.25        4
dtype: int64

In [26]:
df_all = pd.concat([train_data, test_data], axis = 0, ignore_index=True)

In [27]:
df_all = pd.merge(df_all, productDescription, how='left', on='product_uid')

In [35]:
grouped = df_all[~np.isnan(df_all.relevance)].groupby(['search_term'])

In [36]:
ind = 0
for name, group in grouped:
    print 
    print name
    print group[['product_title', 'relevance']]
    ind += 1;
    if ind == 10:
        break


$ hole saw
                                           product_title  relevance
873              Milwaukee Metal Hole Saw Kit (15-Piece)       3.00
1331                   Milwaukee 4 in. Bi-Metal Hole Saw       3.00
14965          Bosch Daredevil Spade Bit Set (10-Pieces)       2.00
24534         PRO-SERIES 19-Piece Hole Saw Set with Case       2.33
27818                       Ryobi Hole Saw Set (6-Piece)       2.33
34053      Milwaukee 3/8 in. Ergo Quick-Change Saw Arbor       2.00
54983        Milwaukee 6-3/8 in. Recessed Light Hole Saw       3.00
70082  Milwaukee 2 in. to 7 in. Dia. Adjustable Hole ...       1.33

. exterior floor stain
                                           product_title  relevance
4105   Glidden Porch and Floor 1-gal. Gloss Polyureth...       3.00
5107   BEHR Premium 1-gal. #902 Slate Gray 1-Part Epo...       2.33
14790            Eagle 1 gal. Rustic Concrete Acid Stain       2.67

.105 trimmer line
                                           product_title  rel

In [19]:
df_all['title_similarity'] = df_all.apply(lambda x : utils.jaccard_similarity(x['product_title'], x['search_term']), axis=1)
df_all['product_desc_similarity'] = df_all.apply(lambda x : utils.jaccard_similarity(x['product_description'], x['search_term']), axis=1)
df_all['description_length'] = df_all.apply(lambda x :len(x.product_description), axis=1)
df_all['search_length'] = df_all.apply(lambda x :len(x.search_term), axis=1)

In [20]:
df_all[['title_similarity', 'product_desc_similarity', 'description_length', 'search_length']]

,title_similarity,product_desc_similarity,description_length,search_length
0,0.200000,0.030000,847,13
1,0.000000,0.030000,847,9
2,0.000000,0.000000,1102,9
3,0.142857,0.087500,694,16
4,0.230769,0.075000,694,18
5,0.062500,0.023490,3202,14
6,0.125000,0.090301,3202,20
7,0.066667,0.047138,3202,10
8,0.100000,0.032967,892,15
9,0.250000,0.048387,474,7


In [49]:
import utils

In [54]:
utils.str_stem("my name is vynal flying flies")

u'my name is vinyl fli fli'